<a href="https://colab.research.google.com/github/Daria-Lytvynenko/ML_course/blob/main/HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, auc, root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from scipy import stats

Mounted at /content/drive


In [2]:
customer_segmentation=pd.read_csv('drive/MyDrive/ML_course/customer_segmentation_train.csv', index_col=0)

In [41]:
customer_segmentation.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,0.0,22,0.0,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,1.0,38,1.0,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,1.0,67,1.0,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,1.0,67,1.0,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,1.0,40,1.0,Entertainment,NaN,High,6.0,Cat_6,A


In [42]:
customer_segmentation.Segmentation.value_counts(normalize=True)

,proportion
Segmentation,
D,0.281111
A,0.244422
C,0.244175
B,0.230293


In [4]:
yes_no_cols=['Ever_Married', 'Graduated']
def code():
  for i in yes_no_cols:
    customer_segmentation[i]=customer_segmentation[i].map({'Yes':1, 'No':0})

In [5]:
code()

In [6]:
customer_segmentation

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,0.0,22,0.0,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,1.0,38,1.0,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,1.0,67,1.0,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,1.0,67,1.0,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,1.0,40,1.0,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...
464018,Male,0.0,22,0.0,NaN,0.0,Low,7.0,Cat_1,D
464685,Male,0.0,35,0.0,Executive,3.0,Low,4.0,Cat_4,D
465406,Female,0.0,33,1.0,Healthcare,1.0,Low,1.0,Cat_6,D


In [7]:
customer_segmentation.Spending_Score.value_counts()

,count
Spending_Score,
Low,4878
Average,1974
High,1216


In [8]:
customer_segmentation.Profession.nunique()

9

In [10]:
null_cols=customer_segmentation.columns[customer_segmentation.isnull().any()].tolist()
for i in null_cols:
  print(customer_segmentation[i].value_counts(normalize=True, dropna=False)*100)


Ever_Married
1.0    57.548339
0.0    40.716411
NaN     1.735250
Name: proportion, dtype: float64
Graduated
1.0    61.576599
0.0    37.456619
NaN     0.966782
Name: proportion, dtype: float64
Profession
Artist           31.184928
Healthcare       16.509668
Entertainment    11.762519
Engineer          8.663857
Doctor            8.527516
Lawyer            7.721864
Executive         7.424393
Marketing         3.619236
Homemaker         3.049083
NaN               1.536936
Name: proportion, dtype: float64
Work_Experience
1.0     29.176996
0.0     28.730788
NaN     10.275161
9.0      5.875062
8.0      5.738721
2.0      3.544869
3.0      3.160635
4.0      3.135845
6.0      2.528508
7.0      2.429351
5.0      2.404561
10.0     0.656916
11.0     0.619732
12.0     0.594943
13.0     0.570154
14.0     0.557759
Name: proportion, dtype: float64
Family_Size
2.0    29.623203
3.0    18.554784
1.0    18.009420
4.0    17.092216
5.0     7.585523
NaN     4.152206
6.0     2.627665
7.0     1.189886
8.0     0.

In [11]:
imputer=SimpleImputer()

In [12]:
null_cols=customer_segmentation.columns[customer_segmentation.isnull().any()].tolist()
for i in null_cols:
  print(customer_segmentation[i].mode())

0    1.0
Name: Ever_Married, dtype: float64
0    1.0
Name: Graduated, dtype: float64
0    Artist
Name: Profession, dtype: object
0    1.0
Name: Work_Experience, dtype: float64
0    2.0
Name: Family_Size, dtype: float64
0    Cat_6
Name: Var_1, dtype: object


In [13]:
  customer_segmentation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   float64
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   float64
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 693.3+ KB


In [14]:
customer_segmentation.describe()

,Ever_Married,Age,Graduated,Work_Experience,Family_Size
count,7928.000000,8068.000000,7990.000000,7239.000000,7733.000000
mean,0.585646,43.466906,0.621777,2.641663,2.850123
std,0.492641,16.711696,0.484974,3.406763,1.531413
min,0.000000,18.000000,0.000000,0.000000,1.000000
25%,0.000000,30.000000,0.000000,0.000000,2.000000
50%,1.000000,40.000000,1.000000,1.000000,3.000000
75%,1.000000,53.000000,1.000000,4.000000,4.000000
max,1.000000,89.000000,1.000000,14.000000,9.000000


In [15]:
X=customer_segmentation.iloc[:, :-1]
y=customer_segmentation.Segmentation

In [26]:
numeric_cols=X.select_dtypes(exclude='object').columns.tolist()
ordinal_col=['Spending_Score']
onehot_cols=['Profession','Var_1']

In [27]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [28]:
numeric_transformer=Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                    ('scaler', StandardScaler())])
onehot_encoder=Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                        ('onehot_enc', OneHotEncoder())])
ordinal_encoder=Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                        ('ordinal_enc', OrdinalEncoder(categories=[['Low', 'Average', 'High']]))])
preprocessor=ColumnTransformer(transformers=[('numeric', numeric_transformer, numeric_cols),
                                             ('onehot', onehot_encoder, onehot_cols),
                                             ('ordinal', ordinal_encoder, ordinal_col)])
model_pipeline=Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', LogisticRegression())])

In [37]:
pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out().tolist()).head(10)

,numeric__Ever_Married,numeric__Age,numeric__Graduated,numeric__Work_Experience,numeric__Family_Size,onehot__Profession_Artist,onehot__Profession_Doctor,onehot__Profession_Engineer,onehot__Profession_Entertainment,onehot__Profession_Executive,...,onehot__Profession_Lawyer,onehot__Profession_Marketing,onehot__Var_1_Cat_1,onehot__Var_1_Cat_2,onehot__Var_1_Cat_3,onehot__Var_1_Cat_4,onehot__Var_1_Cat_5,onehot__Var_1_Cat_6,onehot__Var_1_Cat_7,ordinal__Spending_Score
0,-1.213979,-0.695320,0.766361,1.970880,-1.196771,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.823738,1.703982,0.766361,-0.456381,-0.533884,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,-1.213979,-0.635337,0.766361,-0.456381,0.791890,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.823738,0.264401,0.766361,-0.759788,2.117664,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.823738,-0.935250,-1.304868,1.970880,-1.196771,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.823738,0.084453,0.766361,0.150434,0.129003,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,-1.213979,-0.035512,-1.304868,-0.759788,-1.196771,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,-1.213979,-0.935250,0.766361,-0.456381,0.129003,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,-1.213979,-0.335425,0.766361,0.453842,-1.196771,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.823738,-0.635337,-1.304868,1.364065,1.454777,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?